<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [54]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import module.muresa as muresa

In [8]:
df = pd.read_csv(r'data/df_2.csv')
df.drop('Unnamed: 0',axis = 1,inplace = True)

In [9]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_scaled
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,5.008099,0,-0.994983
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.989578,0,-0.994983
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,5.936639,0,-0.994972
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,4.816242,0,-0.994972
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,4.248354,0,-0.994960


In [13]:
int(np.floor(ones.shape[0]*0.1))

49.0

In [36]:
# 층화추출 10%
ones = df[df['Class']==1].copy()
zeros = df[df['Class']==0].copy()
idx1 = np.random.choice(ones.shape[0],int(np.floor(ones.shape[0]*0.1)),replace = False)
idx2 = np.random.choice(zeros.shape[0],int(np.floor(zeros.shape[0]*0.1)),replace = False)
test_st = pd.concat((ones.iloc[idx1].copy(),zeros.iloc[idx2].copy()),axis = 0)
train_st = pd.concat((ones.drop(ones.index[idx1]),zeros.drop(zeros.index[idx2])) ,axis = 0)

x_train_st, y_train_st, x_test_st, y_test_st = train_st.drop('Class',axis = 1),train_st.Class, test_st.drop('Class',axis = 1),test_st.Class

In [51]:
# train-test 0.3 
x_train,x_test,y_train,y_test = train_test_split(df.drop('Class',axis = 1),df.Class,test_size = 0.3)

In [66]:
# SMOTE 0.4, Train-Test 0.3
x,y = df.drop(columns = 'Class'),df.Class
x_train_sm,x_test_sm, y_train_sm, y_test_sm = muresa.SMOTESampling(x,y, over = 0.4, ratio = 0.3)

In [52]:
glm = LogisticRegression()
glm.fit(x_train,y_train)
y_pred = glm.predict_proba(x_test)
res = evaluation_class(y_test,y_pred)

d:\pl\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
glm2 = LogisticRegression()
glm.fit(x_train_st,y_train_st)
y_pred_st = glm.predict_proba(x_test_st)
res_st = evaluation_class(y_test_st,y_pred_st)

d:\pl\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [101]:
y_train_sm

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

In [103]:
pd.Series(y_test_sm).value_counts()[1]/pd.Series(y_test_sm).value_counts().sum()

0.28367933139607915

In [90]:
y.value_counts()[1]/y.value_counts().sum()

0.001727485630620034

In [61]:
glm3 = LogisticRegression()
glm.fit(x_train_sm,y_train_sm)
y_pred_sm = glm.predict_proba(x_test_sm)
res_sm = evaluation_class(y_test_sm,y_pred_sm)

d:\pl\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [62]:
res_sm

{'confusion matrix': array([[59224,   543],
        [ 2667, 21151]], dtype=int64),
 'acc': 0.9615959801399773,
 'recall': 0.8880258627928458,
 'precision': 0.9749700377984696,
 'f1 score': 0.9294691509931446,
 'average precision': 0.9767970947151287}

In [ ]:
lm

In [43]:
# Evaluation
def evaluation_class(y_true,y_pred,true_idx=1,thresholds = 0.5):
    
    from sklearn import metrics # for result
    import matplotlib.pyplot as plt # for visualization
    
    # tranform y_pred into integer array
    y_pred_val = ([1 if prob>thresholds else 0 for prob in y_pred[:,true_idx]])
    # counfusion matrix
    conf_mat = metrics.confusion_matrix(y_true,y_pred_val)
    
    #acc
    acc = metrics.accuracy_score(y_true,y_pred_val)   
    #recall
    re = metrics.recall_score(y_true,y_pred_val) 
    #precision
    pre = metrics.precision_score(y_true,y_pred_val)
    #f1
    f1 = metrics.f1_score(y_true,y_pred_val)   
    #average precsion
    avg_pre = metrics.average_precision_score(y_true,y_pred[:,true_idx])  
    
    # visualization
#    fig,ax = plt.subplots(1,2,figsize = (15,6))
#     # ROC
#     fpr,tpr,thresholds = metrics.roc_curve(y_true,y_pred[:,true_idx])
#     ax[0].plot(fpr,tpr,color = 'b')
#     ax[0].set_title('ROC Curve',fontsize = 15)
#     ax[0].set_xlabel('1- FP')
#     ax[0].set_ylabel('TP')
    
#     # PR
#     precision,recall,thresholds = metrics.precision_recall_curve(y_true,y_pred[:,true_idx])
#     ax[1].step(recall,precision,color = 'r', where = 'post')
#     ax[1].set_title('PR Curve',fontsize = 15)
#     ax[1].set_xlabel('Recall')
#     ax[1].set_ylabel('Precision')
    
    # 결과 저장
    result_dic = {
        'confusion matrix':conf_mat,
        'acc':acc,
        'recall':re,
        'precision':pre,
        'f1 score':f1,
        'average precision':avg_pre
    }
    return result_dic

In [ ]:
def getKeword():